In [1]:
## libraries
import re
import nltk

In [2]:
## load text
with open('data.txt', 'r') as f:
    data = f.read()

## Tokenization
#comma, dot, apostrophe, colon, semicolon, quotation marks, question, exclamation marks, parenthesis, dollor sign.
#comma, colon, semicolon, question, aprenthesis, double quotation marks, exclamation, dollor sign: 바로 적용
#dot: 문장 마침표에만 적용해야 함. (약어, 소숫점 문제)
#clitics: 's, n't

#1. 우선 다 자르고(clitics 제외)) 모아주기? mwe도 모아주고 등등
#합치기 어떻게? join으로 
#합칠 대상 어떻게 찾지? 중심 토큰과 주변 토큰

punct = re.compile(r'([^A-Za-z\s])') #모든 punctuation
data = re.sub(punct, ' \1 ', data)

#2. 우선 띄어쓰기로 나누고 어절 내에서 토큰화 세부 적용
#세부적용시 조건? 마지막과 처음에서만, r'(?:^|\s)([^A-Za-z\s0-9]+)([A-Za-z0-9]+)(?:$|\s)' > r' \1 \2 ', r'(?:^|\s)([A-Za-z0-9]+)([^A-Za-z\s0-9]+)(?:$|\s)' > r' \1 \2 '
#2번 적용해 줘야 함. 띄어쓰기가 겹치기 때문
#comma: 숫자 문제, 1,000 붙여씀 굳
#loop 돌면서 나눈다?
#abs' 와 같은 문제
#punct: ([^A-Za-z\s0-9]+)
#alpha numeric: ([A-Za-z0-9]+)
#punct 집합을 어떻게 나누지?
#$2.5 안 나뉘어 졌음
#"neuromorphic" 안 나뉘어 졌음..
#n't, 's 같은 clitics 나누어 주어야 함 - can't 는 can n't로 
#U.S.은 왜 안짤렸지? - 내부에 .이 있는 어절 이어서
#.....과 같은 문제 나중에 붙여준다.
#어절 내의 내부구조를 최대한 유지하면서
#Mr. 등등 호칭어의 경우

data = data.split()
data = list(map(lambda x: x, data))





In [3]:
# punct = re.compile(r'(^A-Za-z\s)')

#어절 내의 패턴을 분석 (?:^|\s), (?:$|\s)로 어절을 구분함
front_punct_p = re.compile(r'(?:^|\s)([^A-Za-z\s0-9]+)([A-Za-z0-9\-]+)(?:$|\s)') #ex: "Robots > " Robots
back_punct_p = re.compile(r'(?:^|\s)([A-Za-z0-9\-]+)([^A-Za-z\s0-9]+)(?:$|\s)') #ex: the movie" > the movie "
both_punct_p = re.compile(r'(?:^|\s)([^A-Za-z\s0-9]+)([A-Za-z0-9\-]+)([^A-Za-z\s0-9]+)(?:$|\s)') #ex: "nucleus" > " nucleus "

clitics_p = re.compile(r"(?:^|\s)(\S+?)(n't|'s)(?:$|\s)") #ex: n't, 's 

puncts_p = re.compile(r"(?:^|\s)([^A-Za-z\s0-9])([^A-Za-z\s0-9]+)(?:$|\s)") #ex: )". > ) " .

#완전하지 않음
numerics_with_unit_front_p = re.compile(r"(?:^|\s)([^A-Za-z\s0-9]+)([0-9][\.\,0-9]*)(?:$|\s)") #ex: $2,000 > $ 2,000
numerics_with_unit_back_p = re.compile(r"(?:^|\s)([0-9][\.\,0-9]*)([A-Za-z]]+?)(?:$|\s)") #ex: 1,000won

#2번 적용해 줘야 함. 띄어쓰기가 겹치기 때문, 순서가 중요함. clitics가 먼저 적용될 경우 원하는 결과가 안 나올 수 있다.

data = front_punct_p.sub(r' \1 \2 ', data)
data = back_punct_p.sub(r' \1 \2 ', data)
data = both_punct_p.sub(r' \1 \2 \3 ', data)

data = front_punct_p.sub(r' \1 \2 ', data)
data = back_punct_p.sub(r' \1 \2 ', data)
data = both_punct_p.sub(r' \1 \2 \3 ', data)

data = clitics_p.sub(r' \1 \2 ', data)

while True: #punctuation 연쇄가 없을 때까지 계속 띄어줌
    if not puncts_p.findall(data): #비어있으면, 즉, punctuation 연쇄가 없으면
        break
    data = puncts_p.sub(r' \1 \2 ', data)


data = numerics_with_unit_front_p.sub(r' \1 \2 ', data)
data = numerics_with_unit_back_p.sub(r' \1 \2 ', data)

data = re.sub('\s+', ' ', data) #띄어쓰기 1번(사실상 2번) 이상된 경우를 하나로
data = data.strip()

# data = data.split() ## split 적용 시 띄어쓰기 기준으로 tokenization 진행됨.

with open('data_tokenized.txt', 'w', encoding = 'utf8') as f:
    f.write(data)

print(data)

Three years after its artificial-intelligence engine Watson made its high-profile win on Jeopardy ! , IBM is adapting the technology as it seeks practical commercial uses , an IBM executive explained yesterday at EmTech , a conference organized by MIT Technology Review . The original version of Watson was built around a question-and-answer format optimized for the game show , but that turned out to be " just the first building block " in an emerging artificial-intelligence system , said Mike Rhodin , senior vice president of the IBM Watson Group . Rhodin said IBM is refining Watson to make it more adept at providing the correct answer to a specific question in a specific domain?for example , by learning from previous queries . " Just as [ computer operating system ] platforms emerged in the 1960s , we are seeing the beginnings of that kind of a system emerge in the information age , " he said . In the past year , despite skepticism in some quarters , the company has announced it was ma

In [30]:
## word frequency
#porter stemmer? mwe
##MWE 묶어주기
#대문자 2번 반복 -> 개체명 #San Francisco -> San_Francisco
ne_upper_case_mwe_p = re.compile(r'(?:^|\s)([A-Z]\S+)(?:\s)([A-Z]\S+)(?:$|\s)')
while True:
    if not ne_upper_case_mwe_p.findall(data):
        break
    data = ne_upper_case_mwe_p.sub(r' \1_\2 ', data)
# print(data)

word_tokens = data.split()
word_tokens_lower = list(map(lambda x: x.lower(), word_tokens))

word_freq_list = []
for w in list(set(word_tokens_lower)):
    word_freq_list.append((w,word_tokens_lower.count(w)))

word_freq_list = sorted(word_freq_list, key = lambda x: x[1], reverse = True) #frequency 기준 내림차순 정렬
word_freq_list[:40]

##counter
#word token list 받기

##tuple로 만들기
#set과 list로 구성
#list(set())에서 loop 돌면서 list count 이용. word와 count값을 묶은 tuple로 구성하기.

[(',', 101),
 ('the', 84),
 ('.', 79),
 ('to', 62),
 ('a', 45),
 ('of', 43),
 ('"', 42),
 ('in', 32),
 ('it', 29),
 ('that', 27),
 ('and', 22),
 ('robot', 21),
 ('is', 21),
 ('for', 16),
 ('with', 16),
 ('at', 15),
 ('says', 15),
 ('on', 13),
 ('robots', 10),
 ('its', 10),
 ('by', 10),
 ('could', 9),
 ('an', 9),
 ('are', 9),
 ('more', 9),
 ('would', 8),
 ('as', 8),
 ('company', 8),
 ("'s", 8),
 ('was', 8),
 ('brain_corporation', 8),
 ('has', 8),
 ('said', 8),
 ('watson', 8),
 ('ibm', 7),
 ('see', 7),
 ('out', 7),
 ('fertilizer', 7),
 ('be', 7),
 ('you', 7)]

In [43]:
## Sentence segmentation    
#'.'앞 문맥이 호칭어 종류거나 대문자+. 처럼 개체명의 일부가 아닌 경우에는 문장 종결자
#또 .으로 안 끝나고 \"로 끝날 수도 있음, 현재 데이터셋 에서는 특이한 경우가 나타나지 않았음(호칭어 등)
with open('data.txt', 'r') as f:
    data = f.read()

sent_seg_p = re.compile(r'([\?!\.][\"\']?)(\s+)([\"\']?[A-Z])')

data = sent_seg_p.sub(r'\1\n\3', data) #문장을 개행문자 "\n"로 구분하기 위해 문장별 개행문자 삽입

sent_segs = data.split('\n') #문장을 개행문자 "\n"으로 분리

print(len(sent_segs))
print(sent_segs)


79
['Three years after its artificial-intelligence engine Watson made its high-profile win on Jeopardy!, IBM is adapting the technology as it seeks practical commercial uses, an IBM executive explained yesterday at EmTech, a conference organized by MIT Technology Review.', 'The original version of Watson was built around a question-and-answer format optimized for the game show, but that turned out to be "just the first building block" in an emerging artificial-intelligence system, said Mike Rhodin, senior vice president of the IBM Watson Group.', 'Rhodin said IBM is refining Watson to make it more adept at providing the correct answer to a specific question in a specific domain?for example, by learning from previous queries.', '"Just as [computer operating system] platforms emerged in the 1960s, we are seeing the beginnings of that kind of a system emerge in the information age," he said.', 'In the past year, despite skepticism in some quarters, the company has announced it was making 

In [30]:
' '.join(list('"),'))

'" ) ,'

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\YJW2\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [36]:
## nltk 이용
from nltk.tokenize import word_tokenize, TreebankWordTokenizer, sent_tokenize
import collections
# tokenizer=TreebankWordTokenizer()
# print(tokenizer.tokenize(text))

with open('data.txt', 'r') as f:
    data = f.read()

##tokenization
word_tokens = word_tokenize(data)
# print(word_tokens)

##word_frequency
word_tokens_lower = list(map(lambda x: x.lower(), word_tokens)) 
count = collections.Counter(word_tokens_lower)
most = count.most_common() #빈도 수 순으로 추출
print(most[:40])

##sentence_segmentation
sent_segments = sent_tokenize(data)
print(len(sent_segments))
# print(sent_segments)

[(',', 101), ('the', 86), ('.', 79), ('to', 62), ('a', 45), ('of', 43), ('in', 32), ('it', 29), ('that', 27), ('and', 23), ('is', 22), ('robot', 22), ('``', 21), ("''", 21), ('for', 17), ('with', 16), ('at', 15), ('says', 15), ('watson', 13), ('on', 13), ('its', 10), ('by', 10), ('could', 10), ('robots', 10), ('brain', 10), ('ibm', 9), ('an', 9), ('more', 9), ('are', 9), ('as', 8), ('was', 8), ('said', 8), ('company', 8), ('has', 8), ('would', 8), ('izhikevich', 8), ("'s", 8), ('corporation', 8), ('but', 7), ('out', 7)]
79


In [1]:
# # print(sent_segments)
# for i, s in enumerate(sent_segs):
#     if s != sent_segments[i]:
#         print(i, s)
#         print(i, sent_segments[i])


NameError: name 'sent_segs' is not defined

In [3]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [12]:
from nltk.stem import PorterStemmer 
stemmer = PorterStemmer()
stemmer.stem('ties')

'tie'